<a href="https://colab.research.google.com/github/CarlosHenriqueMatos/neuralNetworksForCompetitions/blob/main/UnetTamanhoNormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow-addons

In [3]:
import tensorflow as tf
import os

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import cv2
from tqdm import tqdm  # Para uma barra de progresso

from skimage.io import imread, imshow
from skimage.transform import resize

import keras.layers as layers
from keras.models import Model
#!pip install keras-adamw

from keras import backend as K

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

In [3]:
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3

In [4]:
# get a list of files in each folder

img_list = os.listdir('/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Training/image/')
mask_list = os.listdir('/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Training/label/')

# create a dataframe
train_df_images = pd.DataFrame(img_list, columns=['image_id'])
train_df_mask = pd.DataFrame(mask_list, columns=['image_id'])

# get a list of files in each folder

validation_img_list = os.listdir('/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Validation/images/')
validation_mask_list = os.listdir('/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Validation/masks/')

# create a dataframe
test_df_images = pd.DataFrame(validation_img_list, columns=['image_id'])
test_df_mask = pd.DataFrame(validation_mask_list, columns=['image_id'])

In [5]:
# Get lists of images and their masks.
train_image_list = list(train_df_images['image_id'])
train_mask_list = list(train_df_mask['image_id'])
test_image_list = list(test_df_images['image_id'])
test_mask_list = list(test_df_mask['image_id'])
# Create empty arrays

X_train = np.zeros((len(train_image_list), IMG_HEIGHT, IMG_WIDTH,3), dtype=np.float32)#You can use float16 or 32, enhance the precision
Y_train = np.zeros((len(train_mask_list), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
X_teste = np.zeros((len(test_image_list), IMG_HEIGHT, IMG_WIDTH,3), dtype=np.float32)
Y_teste = np.zeros((len(test_mask_list), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)

In [6]:
num_images = len(train_image_list)
img_height, img_width, num_channels = 512, 512, 3

# Pré-aloque o array
X_train = np.zeros((num_images, img_height, img_width, num_channels), dtype=np.float32)

for i, img_id in tqdm(enumerate(train_image_list)):
    path_mask = '/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Training/image/' + img_id
    image = cv2.imread(path_mask, 3)  # Usa a flag correta
    X_train[i] = image  # Adiciona diretamente ao array

499it [00:18, 27.11it/s]


In [7]:
num_images = len(train_image_list)
img_height, img_width = 512, 512

# Pré-aloque um array para as máscaras
Y_train = np.zeros((num_images, img_height, img_width), dtype=np.uint8)

for i, img_id in tqdm(enumerate(train_image_list)):
    path_mask = '/content/drive/MyDrive/IFES_2023/AMAZON/Backup_Dataset/Training/label/' + img_id
    image = cv2.imread(path_mask, 0)  # Ler como escala de cinza
    Y_train[i] = image  # Adicionar diretamente ao array

499it [00:03, 138.06it/s]


In [8]:
from keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Lambda
from tensorflow.keras.models import Model

In [9]:

def plota_grafico(historico):
    # figure(figsize = (8, 6))
    ###################### losss
    plt.plot(historico.history['loss'])
    plt.plot(historico.history['val_loss'])
    plt.title('LOSS')
    plt.ylim(ymin = 0,ymax = 1)

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc ='upper left')

    plt.grid()
    plt.show()

    ###################### acuracia

    plt.plot(historico.history['f1_score'])
    plt.plot(historico.history['val_f1_score'])
    plt.title('F1 Score')
    plt.ylim(ymin = 0,ymax = 1)

    plt.ylabel('F1 Score')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc ='upper left')

    plt.grid()
    plt.show()

In [10]:
pip install scikeras


In [11]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, Metric
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [12]:
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        f1_score = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
        return f1_score

    def reset_states(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
#from keras.wrappers.scikit_learn import KerasClassifier


from sklearn.metrics import make_scorer

# Importando os módulos de cálculo de métricas
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import jaccard_score



In [14]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(499, 512, 512, 3) (499, 512, 512)


In [15]:
from sklearn.base import BaseEstimator
import tensorflow as tf

class KerasEstimator(BaseEstimator):
    instance_counter = 0
    def __init__(self, optimizer='adam', learning_rate=0.001, ema_momentum=0.9, kernel_initializer='he_normal'):
        self.learning_rate = learning_rate
        self.ema_momentum = ema_momentum
        self.optimizer = optimizer
        self.kernel_initializer = kernel_initializer
        KerasEstimator.instance_counter += 1  # Initialize the counter


    def fit(self, X, y):
        self.model = self.create_model()

        early_stopper = tf.keras.callbacks.EarlyStopping(
            monitor='accuracy',  # Monitor the f1_score on the validation set
            patience=10,
            verbose=1,
            restore_best_weights=False
        )

        '''checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath="/content/drive/MyDrive/Mestrado/2024_2/Qualification/amazonModels/carlos.h5",
            monitor='accuracy',  # Monitor the f1_score on the validation set
            verbose=1,
            save_best_only=True,
            mode='max'
        )'''

        history = self.model.fit(X, y, batch_size=16, epochs=3, verbose=1,callbacks=[early_stopper])

        print(history.history['accuracy'])
        with open("/content/drive/MyDrive/Mestrado/2024_2/Qualification/amazonModels/treinos1e2.txt", "a") as checkpoint_file:
          checkpoint_file.write("Accuracy History: " + str(history.history['accuracy']) + "\n")

        print("Dados Gravados")

    def predict(self, X):
        return self.model.predict(X)

    def create_model(self):
        #print(self.ema_momentum,self.learning_rate,self.kernel_initializer,self.optimizer)
        inputs = Input((IMG_HEIGHT, IMG_WIDTH, 3))

        s = tf.keras.layers.Lambda(lambda x: x / 65.536) (inputs)

        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (s)
        c1 = tf.keras.layers.Dropout(0.1) (c1)
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (c1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2)) (c1)

        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (p1)
        c2 = tf.keras.layers.Dropout(0.1) (c2)
        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (c2)
        p2 = tf.keras.layers.MaxPooling2D((2, 2)) (c2)

        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (p2)
        c3 = tf.keras.layers.Dropout(0.2) (c3)
        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (c3)
        p3 = tf.keras.layers.MaxPooling2D((2, 2)) (c3)

        c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (p3)
        c4 = tf.keras.layers.Dropout(0.2) (c4)
        c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (c4)
        p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c4)

        c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (p4)
        c5 = tf.keras.layers.Dropout(0.3) (c5)
        c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same') (c5)

        u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
        u6 = tf.keras.layers.Concatenate()([u6, c4])  # Fix the Concatenate usage
        c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(u6)
        c6 = tf.keras.layers.Dropout(0.2)(c6)
        c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(c6)

        u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
        u7 = tf.keras.layers.Concatenate()([u7, c3])  # Fix the Concatenate usage
        c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(u7)
        c7 = tf.keras.layers.Dropout(0.2)(c7)
        c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(c7)

        u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
        u8 = tf.keras.layers.Concatenate()([u8, c2])  # Fix the Concatenate usage
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(u8)
        c8 = tf.keras.layers.Dropout(0.1)(c8)
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(c8)


        u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
        u9 = tf.keras.layers.Concatenate(axis=3)([u9, c1])  # Fix the Concatenate usage
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(u9)
        c9 = tf.keras.layers.Dropout(0.1)(c9)
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=self.kernel_initializer, padding='same')(c9)

        outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)

        model = Model(inputs=[inputs], outputs=[outputs])  # Your model architecture here
        model.compile(optimizer=self.get_optimizer(), loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def get_optimizer(self):
        if self.optimizer == 'adam':
            return tf.keras.optimizers.Adam(learning_rate=self.learning_rate, ema_momentum=self.ema_momentum)
        elif self.optimizer == 'nadam':
            return tf.keras.optimizers.Nadam(learning_rate=self.learning_rate, ema_momentum=self.ema_momentum)
        elif self.optimizer == 'adamw':
            # Implement AdamW optimizer (adjust parameters as needed)
            return tf.keras.optimizers.AdamW(learning_rate=self.learning_rate, ema_momentum=self.ema_momentum)
        else:
            raise ValueError("Unknown optimizer: {}".format(self.optimizer))

# Define hyperparameters for grid search
parameters = {
    'optimizer':['adam'], #['adam','nadam','adamw'],
    'learning_rate': [0.002,0.003,0.004],
    'ema_momentum': [0.975],
    'kernel_initializer': ['he_normal']
}

# Create the wrapped KerasEstimator
#cv = KFold(n_splits=5, shuffle=True, random_state=42)  # Use 5-fold cross-validation
clf = GridSearchCV(estimator = KerasEstimator(), param_grid=parameters,scoring = 'f1',cv=5)
history = clf.fit(X_train, Y_train)
# Fit the grid search

Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5934 - loss: 1.2506
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 522ms/step - accuracy: 0.8655 - loss: 0.3631
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 548ms/step - accuracy: 0.8919 - loss: 0.2967
[0.6829345226287842, 0.869829535484314, 0.8990685939788818]
Dados Gravados
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 888ms/step - accuracy: 0.7106 - loss: 1.6142
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 554ms/step - accuracy: 0.9214 - loss: 0.2372
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 565ms/step - accuracy: 0.9279 - loss: 0.2144
[0.8195514678955078, 0.9209003448486328, 0.9312499165534973]
Dados Gravados
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 779ms/step - accuracy: 0.6050 - loss: 0.7248
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 547ms/step - accuracy: 0.9046 - loss: 0.2798
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 544ms/step - accuracy: 0.9133 - loss: 0.2449
[

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 350ms/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 855ms/step - accuracy: 0.7243 - loss: 0.7020
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 551ms/step - accuracy: 0.8586 - loss: 0.3728
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 560ms/step - accuracy: 0.8974 - loss: 0.2997
[0.807009220123291, 0.8756304383277893, 0.9036256670951843]
Dados Gravados
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 426ms/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 528ms/step - accuracy: 0.6573 - loss: 0.9648
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 547ms/step - accuracy: 0.8848 - loss: 0.3197
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 551ms/step - accuracy: 0.9184 - loss: 0.2323
[0.7587054371833801, 0.8968371748924255, 0.9229679703712463]
Dados Gravados
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 880ms/step - accuracy: 0.7140 - loss: 1.7660
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 545ms/step - accuracy: 0.9040 - loss: 0.2803
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 557ms/

KeyboardInterrupt: 

In [ ]:
# Acesse os melhores parâmetros encontrados no grid_result
best_params = grid_result.best_params_
best_score = grid_result.best_score_

print("Melhores parâmetros encontrados: ", best_params)
print("Melhor precisão encontrada: ", best_score)

In [ ]:
modelo = 0
model = unet()
for train, test in cv.split(X_train,Y_train):
    modelo=modelo+1
    print(modelo)
    fold_no = 1
    acc_per_fold = []
    filepath = "/content/drive/MyDrive/AMAZON/TesteUnet/Unet_Adam_1e-3_10P_he_0.90_/Unet_Adam_1e-3_10P_he_0.90_"+str(modelo)+".h5"

    earlystopper = EarlyStopping(patience=10, verbose=1)

    checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score', verbose=1,
                                  save_best_only=True, mode='max')

    callbacks_list = [earlystopper,checkpoint]

    X = X_train[train]
    Y = Y_train[train]

    Xt = X_train[test]
    Yt = Y_train[test]

    model = unet()
    print("Modelo")
    history=0
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3,ema_momentum=0.9),
                  metrics=[BinaryAccuracy(), F1Score()])

    history = model.fit(X, Y, batch_size=8,validation_data=(Xt, Yt), epochs = 1000,
                    callbacks=callbacks_list,shuffle=False)

    plota_grafico(history)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
dir = "/content/drive/MyDrive/AMAZON/TestesIsolados/"
diretorio = os.listdir(dir)
model = unet()
todasMatrizes=[]
print(diretorio)
for i in range(len(diretorio)):
  model.load_weights(dir+diretorio[i])
  matrizes=[]
  matrizes2=[[0,0],[0,0]]
  amostras = 45
  index=0
  for _ in range(amostras):

      img = cv2.imread('/content/drive/MyDrive/AMAZON/Dataset/Test/image/'+test_image_list[index],3)
      img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
      img = img[np.newaxis, :, :, : ]

      predicted_img = model.predict(img)
      cv2.imwrite('/content/drive/MyDrive/AMAZON/Dataset/Test/'+test_image_list[index], predicted_img)

      #plt.figure(figsize=(12, 12))
      #image = cv2.imread('/content/drive/MyDrive/AMAZON/Dataset/Test/image/'+test_image_list[index], 2)
      #image=image*255
      #plt.subplot(1, 3, 1)
      #plt.imshow(np.squeeze(image))
      #plt.axis('off')
      #plt.title('Original Image')

      #plt.subplot(1, 3, 2)
      #plt.imshow(np.squeeze(cv2.imread('/content/drive/MyDrive/AMAZON/Dataset/Test/mask/'+test_image_list[index],0)))
      #plt.axis('off')
      #plt.title('Original Mask')
      list_arr = cv2.imread('/content/drive/MyDrive/AMAZON/Dataset/Test/mask/'+test_image_list[index],0).tolist()

      #print(list_arr[0])


      #plt.subplot(1, 3, 3)
      #plt.imshow(np.squeeze(predicted_img) >= 0.5 )
      #plt.title('Prediction')
      #plt.axis('off')
      predicted_img=(np.squeeze(predicted_img)>=0.5)


      # assuming the array is named 'arr'
      predicted_img = predicted_img.astype(np.int64)
      list_arr2 = predicted_img.tolist()
      #print(list_arr2[0])
      for i in range(len(list_arr)):
        cm = confusion_matrix(list_arr[i],list_arr2[i])
        if cm.shape == (2, 2):
          matrizes2[0][0]+=cm[0][0]
          matrizes2[0][1]+=cm[0][1]
          matrizes2[1][0]+=cm[1][0]
          matrizes2[1][1]+=cm[1][1]
          teste=np.cumsum(matrizes2)
      #print(teste[3])
      print(matrizes2[0]/teste[3],'\n',matrizes2[1]/teste[3])
      matrizes2=[[0,0],[0,0]]
      matrizes.append(matrizes2)
      plt.show()
      index += 1
      #print(index)
      print(index)
  todasMatrizes.append(matrizes)
  print(todasMatrizes)

In [ ]:
sum_confusion_matrix = np.zeros_like(matrizes[0])

# iterate over all matrices and add them to the sum matrix
for matrizes in todasMatrizes:
  for matrix in matrizes:
      sum_confusion_matrix += matrix
val = sum_confusion_matrix[0][0]+sum_confusion_matrix[0][1]
val1 = sum_confusion_matrix[1][0]+sum_confusion_matrix[1][1]

In [ ]:
TP=sum_confusion_matrix[0][0]
FP=sum_confusion_matrix[0][1]
FN=sum_confusion_matrix[1][0]
TN=sum_confusion_matrix[1][1]
print(TP,FP,FN,TN)

In [ ]:
precisao = TP / (TP + FP)
recall = TP / (TP + FN)
F1_Score = 2 * (precisao * recall) / (precisao + recall)
acuracia = (TP+TN) / (TN+TP+FP+FN)
iou = TP / (TP + FP + FN)
print("Precisão: ",precisao,"\nRecall: ",recall,"\nF1_Score: ",F1_Score,"\nAcuracia: ",acuracia,"\nIoU: ",iou)

In [ ]:
import seaborn as sns
matriz213 =[[TP/(TP + FP),FP/(TP + FP)],[FN/(FN+TN),TN/(FN+TN)]]
sns.heatmap(matriz213, annot=True,fmt=".3f", annot_kws={"size":12}, cmap=plt.cm.Blues)
plt.show()